In [2]:
import os
import sys

# Add root directory (one level up from notebooks/)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

download and install wikiextractor

In [ ]:
# Step 1: Download the ZIP file
!curl -L -o ../wikiextractor.zip https://github.com/qfcy/wikiextractor/archive/refs/heads/master.zip

# Step 2: Extract it
import zipfile
import os

zip_path = "../wikiextractor.zip"
extract_to = "../"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# Step 3: Delete the ZIP file
os.remove(zip_path)

# Step 4: Install Wikiextractor
!pip install -e ../wikiextractor-master


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100 49509  100 49509    0     0  92640      0 --:--:-- --:--:-- --:--:-- 92640


Get wikipedia dump (takes like 2 hours to download)

In [ ]:
os.makedirs("../data/raw", exist_ok=True)
!wget -P ../data/raw https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2

Extract xml data from wikidump

In [ ]:
!python -m wikiextractor.WikiExtractor \
    ../data/raw/enwiki-latest-pages-articles.xml.bz2 \
    -o ../data/raw/extracted_wikidata \
    --no-templates \
    --processes 4

^C


Create json data from wiki-dump

In [ ]:
from utils.data_prep import traverse_directory

input_dir = r'../data/raw/extracted_wikidata'
output_dir = r'../data/processed/wikidata_json'

traverse_directory(input_dir, output_dir)

Processing XML files:   0%|          | 0/19038 [00:00<?, ?file/s]

Create Training data triplets using Pyspark from json wikidata

Had to install winutils for hadoop and pyspark to work on windows locally.

In [ ]:
input_dir = r"../data/processed/wikidata_json"
output_dir = r"../data/processed/triplets/combined"
# output_dir = r"../data/processed/triplets/parts"

In [ ]:
from pyspark.sql import SparkSession
from utils.generate_triplets import create_paragraphs_df, create_triplets

# Initialize Spark session
spark = SparkSession.builder \
    .appName("TripletCreator") \
    .master("local[*]") \
    .config("spark.driver.memory", "24g") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.local.dir", "../spark-temp") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()

df = create_paragraphs_df(spark, input_dir)
triplets = create_triplets(df)
print(f"Writing triplets to Spark part files: {output_dir}")
triplets.coalesce(1).write.mode("overwrite").json(output_dir) # for single json file
# triplets.write.mode("overwrite").json(output_dir) # for multiple json files

# Stop Spark
spark.stop()

Loading JSON input
Writing triplets to Spark part files: ../data/processed/triplets/parts


Delete unneeded files produced by pyspark

In [ ]:
from utils.data_prep import delete_files_in_dir_based_on_ext
delete_files_in_dir_based_on_ext(output_dir, ".json")

Deleted: ../data/processed/triplets/combined\.part-00000-4d87b85e-2b0d-4eda-b097-a980c3c35e5d-c000.json.crc
Deleted: ../data/processed/triplets/combined\._SUCCESS.crc
Preserved: ../data/processed/triplets/combined\part-00000-4d87b85e-2b0d-4eda-b097-a980c3c35e5d-c000.json
Deleted: ../data/processed/triplets/combined\_SUCCESS


Training embedding model

Embed and Index Wikidata

Create faiss index if not already existing

In [ ]:
from utils.faiss_index import create_faiss_index, query_faiss

faiss_path = "../data/embeddings/faiss_index.index"
embedding_path = "../data/embeddings/article_embeddings.npy"

create_faiss_index(faiss_path, embedding_path)